In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('test_data.csv')
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [13]:
import spacy
from spacy.displacy import serve as ss
nlp = spacy.load("ru_core_news_sm")

In [51]:
dict_rus = {'ADJ': 'прилагательное',
    'ADP': 'приставка',
    'ADV': 'наречие',
    'AUX': 'вспомогательный',
    'CCONJ': 'сочинительный союз' ,
    'DET': 'определение',
    'INTJ': 'междометие',
    'NOUN': 'существительное',
    'NUM': 'цифра',
    'PART': 'частица',
    'PRON': 'местоимение',
    'PROPN': 'имя собственное',
    'PUNCT': 'знаки препинания' ,
    'SCONJ': 'подчинительный союз',
    'SYM': 'символ',
    'VERB': 'глагол',
    'X': 'другое'}

In [72]:
text = data['text'][1]
text

'Алло здравствуйте'

In [73]:
doc = nlp(text)
doc

Алло здравствуйте

In [74]:
df = pd.DataFrame(columns = [1, 2, 3, 4, 5])

for token in doc:
    df.loc[df.shape[0]] = [token.text, token.lemma_, token.pos_, token.dep_, token.dep]
df[3] = df[3].map(dict_rus)
df

,1,2,3,4,5
0,Алло,алло,имя собственное,nsubj,429
1,здравствуйте,здравствовать,глагол,ROOT,8206900633647566924


In [78]:
ss(doc)


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
